In [170]:
import numpy as np

## Linear Case

$$
\tau(n) = \frac{1}{n(n-1)}\sum_{u+v=n}(au+bv)\tau(u)\tau(v)
$$
Due to symmetry, we can rewrite the sum as
$$
\tau(n) = \frac{1}{n(n-1)}\sum_{u+v=n}au\tau(u)\tau(v).
$$

In [171]:
def deg_1_array(a_min,a_max,b_min,b_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    
    a_range = np.arange(a_min, a_max+1)
    b_range = np.arange(b_min, b_max+1)
    u_range = np.arange(seq_length)
    v_range = u_range
    
    au_array = np.multiply.outer(a_range,u_range) # a x u
    bv_array = np.multiply.outer(b_range,v_range) # b x v
    
    temp_array = np.add.outer(au_array, bv_array) # a x u x b x v
    
    final_array = np.swapaxes(temp_array, 1,2)
    
    return final_array


def deg_1_sqnc(a_min,a_max,b_min,b_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    
    coeff_array = deg_1_array(a_min,a_max,b_min,b_max,seq_length)
    seq = np.zeros(seq_length)
    seq[1] = 1
    seq_array = np.broadcast_to(seq, (a_len,b_len,seq_length)).copy()
    
    flipped_array = coeff_array[:,:,::-1,:]
    for n in range(2,seq_length):
        diag = np.diagonal(flipped_array, -seq_length+n+1, axis1 = 2, axis2 = 3)[:,:,1:n]
        vec_A = diag*seq_array[:,:,1:n]
        vec_B = seq_array[:,:,n-1:0:-1]
        seq_array[:,:,n] = (1/(n*(n-1)))*np.sum(vec_A*vec_B,axis=2)
    
    return seq_array

In [172]:
deg_1_sqnc(-5,-5,4,4,21)

array([[[ 0.00000000e+00,  1.00000000e+00, -5.00000000e-01,
          2.50000000e-01, -1.25000000e-01,  6.25000000e-02,
         -3.12500000e-02,  1.56250000e-02, -7.81250000e-03,
          3.90625000e-03, -1.95312500e-03,  9.76562500e-04,
         -4.88281250e-04,  2.44140625e-04, -1.22070312e-04,
          6.10351562e-05, -3.05175781e-05,  1.52587891e-05,
         -7.62939453e-06,  3.81469727e-06, -1.90734863e-06]]])

## Quadratic Case

In [173]:
def deg_2_array(a_min,a_max,b_min,b_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    
    a_range = np.arange(a_min, a_max+1)
    b_range = np.arange(b_min, b_max+1)
    u_range = np.arange(seq_length)
    v_range = u_range
    
    au_squared_matrix = np.multiply.outer(a_range, u_range*u_range) # a x u
    au_tensor = np.broadcast_to(au_squared_matrix,(seq_length,b_len,a_len,seq_length)) # k x b x a x k
    #k,b,a,k --> a,b,k,k
    
    bu_matrix = np.multiply.outer(b_range,u_range) # b_len x seq_length
    buv_matrix = np.multiply.outer(bu_matrix,v_range) # b_len x seq_len x seq_len
    buv_tensor = np.broadcast_to(buv_matrix,(a_len,b_len,seq_length,seq_length))
    
    corrected_au_tensor = np.swapaxes(au_tensor, 0,2)
    #corrected_buv_tensor = np.swapaxes(buv_tensor, 2,3)
    
    tensor = corrected_au_tensor + buv_tensor
    
    P_array = np.swapaxes(tensor, 2,3)
    
    return P_array


def deg_2_sqnc(a_min,a_max,b_min,b_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    
    coeff_array = deg_2_array(a_min,a_max,b_min,b_max,seq_length)
    seq = np.zeros(seq_length)
    seq[1] = 1
    seq_array = np.broadcast_to(seq, (a_len,b_len,seq_length)).copy()
    
    flipped_array = coeff_array[:,:,::-1,:]
    for n in range(2,seq_length):
        diag = np.diagonal(flipped_array, -seq_length+n+1, axis1 = 2, axis2 = 3)[:,:,1:n]
        vec_A = diag*seq_array[:,:,1:n]
        vec_B = seq_array[:,:,n-1:0:-1]
        seq_array[:,:,n] = (1/((n*n)*(n-1)))*np.sum(vec_A*vec_B,axis=2)
    
    return seq_array

In [174]:
deg_2_sqnc(-24,-24,36,36,21)

array([[[ 0.,  1.,  3.,  4.,  7.,  6., 12.,  8., 15., 13., 18., 12.,
         28., 14., 24., 24., 31., 18., 39., 20., 42.]]])

## Cubic Case

In [175]:
def deg_3_array(a_min,a_max,b_min,b_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    
    a_range = np.arange(a_min, a_max+1)
    b_range = np.arange(b_min, b_max+1)
    u_range = np.arange(seq_length)
    v_range = u_range
    
    au_squared_matrix = np.multiply.outer(a_range, u_range*u_range*u_range) # a x u^3
    au_tensor = np.broadcast_to(au_squared_matrix,(seq_length,b_len,a_len,seq_length)) # k x b x a x k
    #k,b,a,k --> a,b,k,k
    
    bu_matrix = np.multiply.outer(b_range,u_range*u_range) # b x u^2
    buv_matrix = np.multiply.outer(bu_matrix,v_range) # b x u^2 x v
    buv_tensor = np.broadcast_to(buv_matrix,(a_len,b_len,seq_length,seq_length)) # a x b x k x k
    
    corrected_au_tensor = np.swapaxes(au_tensor, 0,2)
    #corrected_buv_tensor = np.swapaxes(buv_tensor, 2,3)
    
    tensor = corrected_au_tensor + buv_tensor
    
    P_array = np.swapaxes(tensor, 2,3)
    
    return P_array


def deg_3_sqnc(a_min,a_max,b_min,b_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    
    coeff_array = deg_3_array(a_min,a_max,b_min,b_max,seq_length)
    seq = np.zeros(seq_length)
    seq[1] = 1
    seq_array = np.broadcast_to(seq, (a_len,b_len,seq_length)).copy()
    
    flipped_array = coeff_array[:,:,::-1,:]
    for n in range(2,seq_length):
        diag = np.diagonal(flipped_array, -seq_length+n+1, axis1 = 2, axis2 = 3)[:,:,1:n]
        vec_A = diag*seq_array[:,:,1:n]n
        vec_B = seq_array[:,:,n-1:0:-1]
        seq_array[:,:,n] = (1/((n**3)*(n-1)))*np.sum(vec_A*vec_B,axis=2)
    
    return seq_array

In [176]:
deg_3_sqnc(6,6,18,18,21)

array([[[0.00000000e+00, 1.00000000e+00, 3.00000000e+00, 9.00000000e+00,
         2.70000000e+01, 8.10000000e+01, 2.43000000e+02, 7.29000000e+02,
         2.18700000e+03, 6.56100000e+03, 1.96830000e+04, 5.90490000e+04,
         1.77147000e+05, 5.31441000e+05, 1.59432300e+06, 4.78296900e+06,
         1.43489070e+07, 4.30467210e+07, 1.29140163e+08, 3.87420489e+08,
         1.16226147e+09]]])

## Catalan Numbers

In [177]:
def catalan(length):
    catlist = np.zeros(length)
    catlist[1] = 1
    for n in range(2,length):
        vec_A = catlist[1:n]
        vec_B = catlist[n-1:0:-1]
        catlist[n] = np.dot(vec_A,vec_B)

    return catlist

In [178]:
catalan(21)

array([0.00000000e+00, 1.00000000e+00, 1.00000000e+00, 2.00000000e+00,
       5.00000000e+00, 1.40000000e+01, 4.20000000e+01, 1.32000000e+02,
       4.29000000e+02, 1.43000000e+03, 4.86200000e+03, 1.67960000e+04,
       5.87860000e+04, 2.08012000e+05, 7.42900000e+05, 2.67444000e+06,
       9.69484500e+06, 3.53576700e+07, 1.29644790e+08, 4.77638700e+08,
       1.76726319e+09])

## Checking for Geometric Sequences

In [179]:
def array_div(A): # input is 3d array
    orig_sqnc_len = A.shape[2]
    
    shift_1 = A[:,:,2::] # starts each sequence at their first index
    shift_2 = A[:,:,1:-1] # then at their second index
    
    div_array = np.divide(shift_1,shift_2, out=np.full_like(shift_1,np.nan), where=shift_2!=0)
    # puts a zero wherever it had to divide by zero
    
    return div_array

In [180]:
def geom_sqnc_check(A,a_min=0,b_min=0): # need those extra inputs to get the a and b values correct in the tuples
    np.seterr(all="ignore")
   
    quot_array = array_div(A)
    bool_array = (np.abs(quot_array[:,:,:-1] - quot_array[:,:,1:]) < 1e-5).sum(axis=2) == (quot_array.shape[2]-1)

    arrays = np.where(bool_array)
    x_coords = arrays[0]
    y_coords = arrays[1]
    length = len(x_coords)
    coord_list = [(x_coords[n]+a_min,y_coords[n]+b_min,x_coords[n],y_coords[n]) for n in range(length)]
    # I wanted to include the indices as well so it's easier to find in the array
    
    return coord_list

In [181]:
%timeit geom_sqnc_check(deg_3_sqnc(-10,10,-10,10,21),-10,-10)

1.21 ms ± 6.93 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [182]:
array_div(deg_1_sqnc(1000,1000,-327,-327,21)) # a = 1000, b = -327

array([[[336.5, 336.5, 336.5, 336.5, 336.5, 336.5, 336.5, 336.5, 336.5,
         336.5, 336.5, 336.5, 336.5, 336.5, 336.5, 336.5, 336.5, 336.5,
         336.5]]])

It looks like the linear case *only* produces geometric sequences!

In [183]:
geom_sqnc_check(deg_2_sqnc(-10,10,-10,10,21),-10,-10)

[(-10, -10, 0, 0),
 (-9, -9, 1, 1),
 (-8, -8, 2, 2),
 (-7, -7, 3, 3),
 (-6, -6, 4, 4),
 (-5, -5, 5, 5),
 (-4, -4, 6, 6),
 (-3, -3, 7, 7),
 (-2, -2, 8, 8),
 (-1, -1, 9, 9),
 (1, 1, 11, 11),
 (2, 2, 12, 12),
 (3, 3, 13, 13),
 (4, 4, 14, 14),
 (5, 5, 15, 15),
 (6, 6, 16, 16),
 (7, 7, 17, 17),
 (8, 8, 18, 18),
 (9, 9, 19, 19),
 (10, 10, 20, 20)]

In [184]:
array_div(deg_2_sqnc(-10,10,-10,10,21))[14,14]

array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2.])

In [185]:
array_div(deg_2_sqnc(-10,10,-10,10,21))[16,16]

array([3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3.])

In [186]:
array_div(deg_2_sqnc(-10,10,-10,10,21))[16,4]

array([ 0., nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan])

Looks like the quadratic function produces a geometric sequence for each ordered pair $(a,b) = (a,a)$ and sequences of zeros for $(a,b) = (a,-a)$, $a \in \mathbb{Z}$.

In [187]:
geom_sqnc_check(deg_3_sqnc(-30,30,-50,50,21),-30,-50)

[(-16, -48, 14, 2),
 (-15, -45, 15, 5),
 (-14, -42, 16, 8),
 (-13, -39, 17, 11),
 (-12, -36, 18, 14),
 (-11, -33, 19, 17),
 (-10, -30, 20, 20),
 (-9, -27, 21, 23),
 (-8, -24, 22, 26),
 (-7, -21, 23, 29),
 (-6, -18, 24, 32),
 (-5, -15, 25, 35),
 (-4, -12, 26, 38),
 (-3, -9, 27, 41),
 (-2, -6, 28, 44),
 (-1, -3, 29, 47),
 (1, 3, 31, 53),
 (2, 6, 32, 56),
 (3, 9, 33, 59),
 (4, 12, 34, 62),
 (5, 15, 35, 65),
 (6, 18, 36, 68),
 (7, 21, 37, 71),
 (8, 24, 38, 74),
 (9, 27, 39, 77),
 (10, 30, 40, 80),
 (11, 33, 41, 83),
 (12, 36, 42, 86),
 (13, 39, 43, 89),
 (14, 42, 44, 92),
 (15, 45, 45, 95),
 (16, 48, 46, 98)]

In [188]:
deg_3_sqnc(-24,-24,48,48,21) # finding our original sigma_n in the cubics

array([[[ 0.,  1.,  3.,  4.,  7.,  6., 12.,  8., 15., 13., 18., 12.,
         28., 14., 24., 24., 31., 18., 39., 20., 42.]]])

## Some Ideas

After finding a function that finds the geometric sequences, try dividing everything by sequences of catalan numbers or sequences of sums of factors, then put it through the function. This way we can pinpoint the combo-sequences.

### Finding Combo Sequences with $\sigma_n$

In [189]:
divsigma = deg_2_sqnc(-40,40,-40,40,21)
sigma = deg_2_sqnc(-24,-24,36,36,21)
divsigma[:,:,1::] = divsigma[:,:,1::]/sigma[:,:,1::]

geom_sqnc_check(divsigma,-40,-40)

[(-26, 39, 14, 79),
 (-24, 36, 16, 76),
 (-22, 33, 18, 73),
 (-20, 30, 20, 70),
 (-18, 27, 22, 67),
 (-16, 24, 24, 64),
 (-14, 21, 26, 61),
 (-12, 18, 28, 58),
 (-10, 15, 30, 55),
 (-8, 12, 32, 52),
 (-6, 9, 34, 49),
 (-4, 6, 36, 46),
 (-2, 3, 38, 43),
 (2, -3, 42, 37),
 (4, -6, 44, 34),
 (6, -9, 46, 31),
 (8, -12, 48, 28),
 (10, -15, 50, 25),
 (12, -18, 52, 22),
 (14, -21, 54, 19),
 (16, -24, 56, 16),
 (18, -27, 58, 13),
 (20, -30, 60, 10),
 (22, -33, 62, 7),
 (24, -36, 64, 4),
 (26, -39, 66, 1)]

First look at possible combo-sequences:

In [190]:
array_div(divsigma)[14,79] # a = -26, b = 39

array([1.08333333, 1.08333333, 1.08333333, 1.08333333, 1.08333333,
       1.08333333, 1.08333333, 1.08333333, 1.08333333, 1.08333333,
       1.08333333, 1.08333333, 1.08333333, 1.08333333, 1.08333333,
       1.08333333, 1.08333333, 1.08333333, 1.08333333])

Looks like $\big\{\left(\frac{13}{12}\right)^n\big\}$ for $a = -26$ and $b = 39$. Now to look at the original sequence in that place:

In [191]:
deg_2_sqnc(-40,40,-40,40,21)[14,79]

array([  0.        ,   1.        ,   3.25      ,   4.69444444,
         8.89988426,   8.26417824,  17.90571952,  12.93190854,
        26.26793923,  24.66267628,  36.99401441,  26.7178993 ,
        67.53691212,  36.58249406,  67.93891755,  73.60049401,
       102.98958016,  64.78376816, 152.06190027,  84.47883348,
       192.18934618])

In [192]:
array_div(deg_2_sqnc(-40,40,-40,40,21))[52,22]

array([-1.5       , -0.66666667, -0.875     , -0.42857143, -1.        ,
       -0.33333333, -0.9375    , -0.43333333, -0.69230769, -0.33333333,
       -1.16666667, -0.25      , -0.85714286, -0.5       , -0.64583333,
       -0.29032258, -1.08333333, -0.25641026, -1.05      ])

In [193]:
array_div(divsigma)[52,22]

array([-0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5,
       -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5])

Looks pretty evident that $a = 12$, $b = -18$ $\Rightarrow$ $\{-0.5^n\}\times\sigma(n)$. Now to find some more:

In [194]:
array_div(divsigma)[22,67]

array([0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75,
       0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75])

$a = -18$, $b = 27$ $\Rightarrow$ $\{0.75^n\}\times\sigma(n)$.

In [195]:
array_div(divsigma)[34, 49]

array([0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25,
       0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25])

$a = -6, b = 9$ $\Rightarrow$ $\{0.25^n\}\times\sigma(n)$

In [196]:
array_div(divsigma)[16, 76] # this is just where sigma_n would be

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1.])

In [197]:
array_div(divsigma)[38,43] # 0.08333333... = 1/12

array([0.08333333, 0.08333333, 0.08333333, 0.08333333, 0.08333333,
       0.08333333, 0.08333333, 0.08333333, 0.08333333, 0.08333333,
       0.08333333, 0.08333333, 0.08333333, 0.08333333, 0.08333333,
       0.08333333, 0.08333333, 0.08333333, 0.08333333])

$a = -2, \ b = 3 \ \Rightarrow \ \big\{\left(\frac{1}{12}\right)^n\big\}\times\sigma(n)$. Didn't want to show up from the function because of floating point inconsistencies.

In [198]:
array_div(divsigma)[28,58]

array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5])

$a = -12$, $b = 18$ $\Rightarrow$ $\{0.5^n\}\times\sigma(n)$

In [199]:
array_div(divsigma)[58,13]

array([-0.75, -0.75, -0.75, -0.75, -0.75, -0.75, -0.75, -0.75, -0.75,
       -0.75, -0.75, -0.75, -0.75, -0.75, -0.75, -0.75, -0.75, -0.75,
       -0.75])

$a = 18$, $b = -27$ $\Rightarrow$ $\{-0.75^n\}\times\sigma(n)$.

In [200]:
array_div(divsigma)[36,46] # 0.166666... = 1/6

array([0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
       0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
       0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
       0.16666667, 0.16666667, 0.16666667, 0.16666667])

$a = -4, \ b = 6 \ \Rightarrow \big\{\left(\frac{1}{6}\right)^n\big\}\times\sigma(n)$

What I'm seeing is that, for $\tau(n) = \frac{1}{n^2(n-1)}\sum_{u+v=n}(au^2+buv)\tau(u)\tau(v)$, $(a,b)=(a,a)$ produces a geometric sequence $\{\rho^n\}$ for $\rho \in \mathbb{Q}$, $(a,b) = (-24,36)$ produces $\sigma(n)$, and $(a,b) = (-2k,3k)$ for $k \in \mathbb{Z}$ produces some sequence $\{\rho^n\}\times\sigma(n)$. The next step would be to connect $a$ and $\rho$ as well as $k$ and $\rho$.

# Next Stuff

Play with other denominators, like $\frac{1}{n^3}$ or $\frac{1}{n(n-1)(n-2)}$. Also extend search to higher degree polynomials - quartics, quintics, etc. Put together a function that checks for sequences of integers.

### Integer Sequence Checker

In [201]:
def int_sqnc_check(A,a_min=0,b_min=0):
    bool_array = (np.abs(A - np.around(A)) < 1e-5).sum(axis=2) >= (A.shape[2]-1)
    
    arrays = np.where(bool_array)
    x_coords = arrays[0]
    y_coords = arrays[1]
    length = len(x_coords)
    coord_list = [(x_coords[n]+a_min,y_coords[n]+b_min,x_coords[n],y_coords[n]) for n in range(length)]
    
    return coord_list

In [206]:
A = deg_2_sqnc(-30,30,-50,50,21)
int_sqnc_check(A,-30,-50)

[(-30, 30, 0, 80),
 (-29, 29, 1, 79),
 (-28, -28, 2, 22),
 (-28, 28, 2, 78),
 (-27, 27, 3, 77),
 (-26, 26, 4, 76),
 (-25, 25, 5, 75),
 (-24, -24, 6, 26),
 (-24, 24, 6, 74),
 (-24, 36, 6, 86),
 (-23, 23, 7, 73),
 (-22, -22, 8, 28),
 (-22, 22, 8, 72),
 (-21, 21, 9, 71),
 (-20, -20, 10, 30),
 (-20, 20, 10, 70),
 (-19, 19, 11, 69),
 (-18, 18, 12, 68),
 (-17, 17, 13, 67),
 (-16, -16, 14, 34),
 (-16, 16, 14, 66),
 (-15, 15, 15, 65),
 (-14, 14, 16, 64),
 (-13, 13, 17, 63),
 (-12, -12, 18, 38),
 (-12, 12, 18, 62),
 (-11, 11, 19, 61),
 (-10, -10, 20, 40),
 (-10, 10, 20, 60),
 (-9, 9, 21, 59),
 (-8, -8, 22, 42),
 (-8, 8, 22, 58),
 (-7, 7, 23, 57),
 (-6, -6, 24, 44),
 (-6, 6, 24, 56),
 (-5, 5, 25, 55),
 (-4, -4, 26, 46),
 (-4, 4, 26, 54),
 (-3, 3, 27, 53),
 (-2, -2, 28, 48),
 (-2, 2, 28, 52),
 (-1, 1, 29, 51),
 (0, 0, 30, 50),
 (1, -1, 31, 49),
 (2, -2, 32, 48),
 (2, 2, 32, 52),
 (3, -3, 33, 47),
 (4, -4, 34, 46),
 (4, 4, 34, 54),
 (5, -5, 35, 45),
 (6, -6, 36, 44),
 (6, 6, 36, 56),
 (7, -7, 37, 

In [207]:
A[54,14]

array([  0.,   1.,  -3.,   4.,  -7.,   6., -12.,   8., -15.,  13., -18.,
        12., -28.,  14., -24.,  24., -31.,  18., -39.,  20., -42.])

In [208]:
B = deg_3_sqnc(-50,50,-50,50,21)
int_sqnc_check(B,-50,-50)

[(-50, 50, 0, 100),
 (-49, 49, 1, 99),
 (-48, 48, 2, 98),
 (-47, 47, 3, 97),
 (-46, 46, 4, 96),
 (-45, 45, 5, 95),
 (-44, 44, 6, 94),
 (-43, 43, 7, 93),
 (-42, 42, 8, 92),
 (-41, 41, 9, 91),
 (-40, 40, 10, 90),
 (-39, 39, 11, 89),
 (-38, 38, 12, 88),
 (-37, 37, 13, 87),
 (-36, 36, 14, 86),
 (-35, 35, 15, 85),
 (-34, 34, 16, 84),
 (-33, 33, 17, 83),
 (-32, 32, 18, 82),
 (-31, 31, 19, 81),
 (-30, 30, 20, 80),
 (-29, 29, 21, 79),
 (-28, 28, 22, 78),
 (-27, 27, 23, 77),
 (-26, 26, 24, 76),
 (-25, 25, 25, 75),
 (-24, 24, 26, 74),
 (-24, 48, 26, 98),
 (-23, 23, 27, 73),
 (-22, 22, 28, 72),
 (-21, 21, 29, 71),
 (-20, 20, 30, 70),
 (-19, 19, 31, 69),
 (-18, 18, 32, 68),
 (-17, 17, 33, 67),
 (-16, -48, 34, 2),
 (-16, 16, 34, 66),
 (-15, 15, 35, 65),
 (-14, 14, 36, 64),
 (-13, 13, 37, 63),
 (-12, -36, 38, 14),
 (-12, 12, 38, 62),
 (-11, 11, 39, 61),
 (-10, -30, 40, 20),
 (-10, 10, 40, 60),
 (-9, 9, 41, 59),
 (-8, -24, 42, 26),
 (-8, 8, 42, 58),
 (-7, 7, 43, 57),
 (-6, -18, 44, 32),
 (-6, 6, 44, 

In [209]:
B[26,98]

array([ 0.,  1.,  3.,  4.,  7.,  6., 12.,  8., 15., 13., 18., 12., 28.,
       14., 24., 24., 31., 18., 39., 20., 42.])

In [210]:
len(B.shape)

3

### Quartic Case

Now we're letting $P(u,v) = au^4 + bu^3v + cu^2v^2$. This instantly complicates things, since I'll need to include a new axis in my matrix, but it should involve the same idea. In the end, the ordered triplet $(a,b,c) = (-24,24,48)$ should yield our $\sigma(n)$.

I'll also need to create new sequence checkers (or add features to the current ones) for each of these to account for the extra axis.

In [304]:
def deg_4_array(a_min,a_max,b_min,b_max,c_min,c_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    c_len = c_max+1 - c_min
    
    a_range = np.arange(a_min, a_max+1)
    b_range = np.arange(b_min, b_max+1)
    c_range = np.arange(c_min, c_max+1)
    u_range = np.arange(seq_length)
    v_range = u_range
    
    au_matrix = np.multiply.outer(a_range, u_range**4) # a x u^4
    au_tensor = np.broadcast_to(au_matrix,(seq_length,b_len,c_len,a_len,seq_length)) # k x b x c x a x u^4
    #v,b,c,a,u --> a,b,c,v,u --> a,b,c,u,v
    
    bu_matrix = np.multiply.outer(b_range,u_range**3) # b x u^3
    buv_matrix = np.multiply.outer(bu_matrix,v_range) # b x u^3 x v
    buv_tensor = np.broadcast_to(buv_matrix,(a_len,c_len,b_len,seq_length,seq_length)) # a x c x b x u^3 x v
    # a,c,b,u,v --> a,b,c,u,v
    
    cu_matrix = np.multiply.outer(c_range,u_range**2) # c x u^2
    cuv_matrix = np.multiply.outer(cu_matrix,v_range**2) # c x u^2 x v^2
    cuv_tensor = np.broadcast_to(cuv_matrix,(a_len,b_len,c_len,seq_length,seq_length)) # a x b x c x u^2 x v^2
    
    corrected_au_tensor = np.swapaxes(np.swapaxes(au_tensor, 3,4),0,4)
    corrected_buv_tensor = np.swapaxes(buv_tensor, 1,2)
    
    tensor = corrected_au_tensor + corrected_buv_tensor + cuv_tensor
    
    #P_array = np.swapaxes(tensor, 3,4)
    
    return tensor


def deg_4_sqnc(a_min,a_max,b_min,b_max,c_min,c_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    c_len = c_max+1 - c_min
    
    coeff_array = deg_4_array(a_min,a_max,b_min,b_max,c_min,c_max,seq_length)
    seq = np.zeros(seq_length)
    seq[1] = 1
    seq_array = np.broadcast_to(seq, (a_len,b_len,c_len,seq_length)).copy()
    
    flipped_array = coeff_array[:,:,:,::-1,:]
    for n in range(2,seq_length):
        diag = np.diagonal(flipped_array, -seq_length+n+1, axis1 = 3, axis2 = 4)[:,:,:,1:n]
        vec_A = diag*seq_array[:,:,:,1:n]
        vec_B = seq_array[:,:,:,n-1:0:-1]
        seq_array[:,:,:,n] = (1/((n**4)*(n-1)))*np.sum(vec_A*vec_B,axis=3)
    
    return seq_array

In [305]:
deg_4_sqnc(-24,-24,24,24,48,48,21)

array([[[[ 0.,  1.,  3.,  4.,  7.,  6., 12.,  8., 15., 13., 18., 12.,
          28., 14., 24., 24., 31., 18., 39., 20., 42.]]]])

### Quintic Case

Home is at $(a,b,c) = (-24,0,120)$.

In [307]:
def deg_5_array(a_min,a_max,b_min,b_max,c_min,c_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    c_len = c_max+1 - c_min
    
    a_range = np.arange(a_min, a_max+1)
    b_range = np.arange(b_min, b_max+1)
    c_range = np.arange(c_min, c_max+1)
    u_range = np.arange(seq_length)
    v_range = u_range
    
    au_matrix = np.multiply.outer(a_range, u_range**5) # a x u^5
    au_tensor = np.broadcast_to(au_matrix,(seq_length,b_len,c_len,a_len,seq_length)) # k x b x c x a x u^4
    #v,b,c,a,u --> a,b,c,v,u --> a,b,c,u,v
    
    bu_matrix = np.multiply.outer(b_range,u_range**4) # b x u^4
    buv_matrix = np.multiply.outer(bu_matrix,v_range) # b x u^4 x v
    buv_tensor = np.broadcast_to(buv_matrix,(a_len,c_len,b_len,seq_length,seq_length)) # a x c x b x u^3 x v
    # a,c,b,u,v --> a,b,c,u,v
    
    cu_matrix = np.multiply.outer(c_range,u_range**3) # c x u^3
    cuv_matrix = np.multiply.outer(cu_matrix,v_range**2) # c x u^3 x v^2
    cuv_tensor = np.broadcast_to(cuv_matrix,(a_len,b_len,c_len,seq_length,seq_length)) # a x b x c x u^3 x v^2
    
    corrected_au_tensor = np.swapaxes(np.swapaxes(au_tensor, 3,4),0,4)
    corrected_buv_tensor = np.swapaxes(buv_tensor, 1,2)
    
    tensor = corrected_au_tensor + corrected_buv_tensor + cuv_tensor
    
    #P_array = np.swapaxes(tensor, 3,4)
    
    return tensor


def deg_5_sqnc(a_min,a_max,b_min,b_max,c_min,c_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    c_len = c_max+1 - c_min
    
    coeff_array = deg_5_array(a_min,a_max,b_min,b_max,c_min,c_max,seq_length)
    seq = np.zeros(seq_length)
    seq[1] = 1
    seq_array = np.broadcast_to(seq, (a_len,b_len,c_len,seq_length)).copy()
    
    flipped_array = coeff_array[:,:,:,::-1,:]
    for n in range(2,seq_length):
        diag = np.diagonal(flipped_array, -seq_length+n+1, axis1 = 3, axis2 = 4)[:,:,:,1:n]
        vec_A = diag*seq_array[:,:,:,1:n]
        vec_B = seq_array[:,:,:,n-1:0:-1]
        seq_array[:,:,:,n] = (1/((n**5)*(n-1)))*np.sum(vec_A*vec_B,axis=3)
    
    return seq_array

In [311]:
deg_5_sqnc(-24,-24,0,0,120,120,21)

array([[[[ 0.,  1.,  3.,  4.,  7.,  6., 12.,  8., 15., 13., 18., 12.,
          28., 14., 24., 24., 31., 18., 39., 20., 42.]]]])

Gotta update the dang sequence checkers and play with the $\frac{1}{n^p(n-1)}$ thing.